In [1]:
from mtANN_model import *
import numpy as np
from collections import Counter

# load query data

In [2]:
data_collection = ["v2","v3","cl","dr","id","sw","sm2"]
features = ["de","dv","dd","dp","bi","gc","disp","vst"]
dt = 1 
unseen_ct = "B cell"

target = data_collection[dt]
print("query dataset is {}, the unseen cell type is {}".format(target,unseen_ct))
target_dataset = {}
target_dataset["expression"] = pd.read_csv("./datasets/pbmc/{}.csv".format(target), header=0, index_col=0)
target_dataset["cell_type"] = pd.read_csv("./datasets/pbmc/{}_label.csv".format(target), header=0, index_col=False).x.values
target_dataset["cell_type"][np.where(target_dataset["cell_type"] == unseen_ct)[0]] = "unassigned"
unseen_prop = len(np.where(target_dataset["cell_type"] == "unassigned")[0])/len(target_dataset["cell_type"])
print("The proportion of unseen cell types is {}".format(unseen_prop))

query dataset is v3, the unseen cell type is B cell
The proportion of unseen cell types is 0.1130718954248366


# load reference datasets and obtain reference subsets

In [3]:
epr_s = list()
label_s = list()
epr_t = list()
label_t = list()
for ds in set(np.arange(len(data_collection))).difference(set([dt])):
    for g in range(len(features)):
        source = data_collection[ds]
        gene = features[g]
        print("source is {}, gene selection method is {}".format(source,gene))
        source_dataset = {}
        source_dataset["expression"] = pd.read_csv("./datasets/pbmc/{}.csv".format(source), header=0, index_col=0)
        source_dataset["cell_type"] = pd.read_csv("./datasets/pbmc/{}_label.csv".format(source), header=0, index_col=False).x.values
        target_dataset["expression"] = pd.read_csv("./datasets/pbmc/{}.csv".format(target), header=0, index_col=0)
        
        source_dataset["expression"] = source_dataset["expression"].div(source_dataset["expression"].apply(lambda x: x.sum(), axis=1), axis=0) * 10000
        target_dataset["expression"] = target_dataset["expression"].div(target_dataset["expression"].apply(lambda x: x.sum(), axis=1), axis=0) * 10000
        
        select_gene_ref = pd.read_csv("./datasets/pbmc/pbmc_genes/_{}_{}_.csv".format(source, gene), header=0, index_col=0)["x"]
        # select_gene_ref = select_gene(source_dataset["expression"], source_dataset["cell_type"], gene)
        gene_select = set(select_gene_ref).intersection(set(target_dataset["expression"].columns))
        
        source_dataset["expression"] = source_dataset["expression"][np.array(list(gene_select))]
        target_dataset["expression"] = target_dataset["expression"][np.array(list(gene_select))]

        id = np.where(source_dataset["cell_type"] == unseen_ct)[0]
        id_name = source_dataset["expression"].index[id]
        source_dataset["expression"] = source_dataset["expression"].drop(id_name, axis = 0)
        source_dataset["cell_type"] = np.delete(source_dataset["cell_type"],id)

        data_s, data_t = preprocess(source_dataset, target_dataset)
        epr_s.append(data_s["expression"])
        epr_t.append(data_t["expression"])
        label_t.append(data_t["cell_type"])
        label_s.append(data_s["cell_type"])

source is v2, gene selection method is de
source is v2, gene selection method is dv
source is v2, gene selection method is dd
source is v2, gene selection method is dp
source is v2, gene selection method is bi
source is v2, gene selection method is gc
source is v2, gene selection method is disp
source is v2, gene selection method is vst
source is cl, gene selection method is de
source is cl, gene selection method is dv
source is cl, gene selection method is dd
source is cl, gene selection method is dp
source is cl, gene selection method is bi
source is cl, gene selection method is gc
source is cl, gene selection method is disp
source is cl, gene selection method is vst
source is dr, gene selection method is de
source is dr, gene selection method is dv
source is dr, gene selection method is dd
source is dr, gene selection method is dp
source is dr, gene selection method is bi
source is dr, gene selection method is gc
source is dr, gene selection method is disp
source is dr, gene selecti

# train deep classification models and annotate query data

In [6]:
mid_annotation, final_annotation, m, threshold = mtANN(expression_s = epr_s, label_s=label_s, expression_t=epr_t, threshold="default", gene_select="none")

training 1-th classification model
training 2-th classification model
training 3-th classification model
training 4-th classification model
training 5-th classification model
training 6-th classification model
training 7-th classification model
training 8-th classification model
training 9-th classification model
training 10-th classification model
training 11-th classification model
training 12-th classification model
training 13-th classification model
training 14-th classification model
training 15-th classification model
training 16-th classification model
training 17-th classification model
training 18-th classification model
training 19-th classification model
training 20-th classification model
training 21-th classification model
training 22-th classification model
training 23-th classification model
training 24-th classification model
training 25-th classification model
training 26-th classification model
training 27-th classification model
training 28-th classification model
t

# calculate annotation accuracy

In [7]:
accuracy = sum(final_annotation[:,0] == target_dataset["cell_type"])/final_annotation.shape[0]
accuracy_unseen = len(set(np.where(target_dataset["cell_type"] == "unassigned")[0]).intersection(set(np.where(final_annotation[:,0] == "unassigned")[0])))/len(np.where(target_dataset["cell_type"] == "unassigned")[0])
print("The annotation accuracy with the default threshold is {}".format(accuracy))
print("The annotation accuracy of unseen cell type with the default threshold is {}".format(accuracy_unseen))

The annotation accuracy with the default threshold is 0.9081699346405229
The annotation accuracy of unseen cell type with the default threshold is 0.8208092485549133
